In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("shusrith/machine-trainslation")

# print("Path to dataset files:", path)

In [2]:
# !ls $path/EnglishOrSpanish

In [1]:
import pandas as pd
from sklearn.utils import shuffle
df = pd.read_csv(f"EnglishOrSpanish/output_joint1.csv")
df = shuffle(df)
df

,English,Spanish
37238,"[2, 290, 506, 302, 4187, 272, 382, 224, 3, 0, ...","[2, 927, 302, 590, 11051, 632, 224, 3, 0, 0, 0..."
16313,"[2, 404, 334, 479, 977, 224, 3, 0, 0, 0, 0, 0,...","[2, 4794, 318, 352, 644, 224, 3, 0, 0, 0, 0, 0..."
119558,"[2, 272, 4285, 265, 4942, 10980, 391, 300, 293...","[2, 470, 13559, 410, 5529, 2502, 14723, 322, 3..."
77063,"[2, 290, 1581, 265, 8717, 1092, 9923, 224, 3, ...","[2, 290, 5329, 323, 9350, 387, 4920, 294, 352,..."
117062,"[2, 1041, 6743, 270, 1038, 1245, 2274, 7257, 3...","[2, 1458, 294, 1147, 954, 328, 1319, 2542, 224..."
...,...,...
86927,"[2, 818, 803, 982, 611, 5959, 609, 224, 3, 0, ...","[2, 831, 328, 311, 1464, 387, 321, 2518, 27400..."
50420,"[2, 300, 2077, 2324, 391, 582, 224, 3, 0, 0, 0...","[2, 331, 2743, 2350, 8040, 224, 3, 0, 0, 0, 0,..."
83302,"[2, 900, 803, 921, 843, 498, 370, 224, 3, 0, 0...","[2, 535, 2984, 7162, 704, 1323, 224, 3, 0, 0, ..."
64192,"[2, 290, 20104, 300, 1182, 272, 369, 224, 3, 0...","[2, 290, 372, 26803, 331, 874, 265, 369, 224, ..."


In [3]:
import ast

df["English"] = df["English"].apply(ast.literal_eval)
df["Spanish"] = df["Spanish"].apply(ast.literal_eval)
df

,English,Spanish
37238,"[2, 290, 506, 302, 4187, 272, 382, 224, 3, 0, ...","[2, 927, 302, 590, 11051, 632, 224, 3, 0, 0, 0..."
16313,"[2, 404, 334, 479, 977, 224, 3, 0, 0, 0, 0, 0,...","[2, 4794, 318, 352, 644, 224, 3, 0, 0, 0, 0, 0..."
119558,"[2, 272, 4285, 265, 4942, 10980, 391, 300, 293...","[2, 470, 13559, 410, 5529, 2502, 14723, 322, 3..."
77063,"[2, 290, 1581, 265, 8717, 1092, 9923, 224, 3, ...","[2, 290, 5329, 323, 9350, 387, 4920, 294, 352,..."
117062,"[2, 1041, 6743, 270, 1038, 1245, 2274, 7257, 3...","[2, 1458, 294, 1147, 954, 328, 1319, 2542, 224..."
...,...,...
86927,"[2, 818, 803, 982, 611, 5959, 609, 224, 3, 0, ...","[2, 831, 328, 311, 1464, 387, 321, 2518, 27400..."
50420,"[2, 300, 2077, 2324, 391, 582, 224, 3, 0, 0, 0...","[2, 331, 2743, 2350, 8040, 224, 3, 0, 0, 0, 0,..."
83302,"[2, 900, 803, 921, 843, 498, 370, 224, 3, 0, 0...","[2, 535, 2984, 7162, 704, 1323, 224, 3, 0, 0, ..."
64192,"[2, 290, 20104, 300, 1182, 272, 369, 224, 3, 0...","[2, 290, 372, 26803, 331, 874, 265, 369, 224, ..."


In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset

train = TensorDataset(
    torch.tensor(df["English"][:100000].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][:100000].tolist(), dtype=torch.long),
)
test = TensorDataset(
    torch.tensor(df["English"][100000:].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][100000:].tolist(), dtype=torch.long),
)
train_loader = DataLoader(train, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test, batch_size=256, shuffle=False, drop_last=True)

In [5]:
import json
with open(f"EnglishOrSpanish/vocab.json", "r") as f:
    vocab = json.load(f)

In [6]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=4, dropout=0.3):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=14453)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)

        self.layer_norm = nn.LayerNorm(2 * hidden_size)
        self.hidden_norm = nn.LayerNorm(hidden_size)
        self.cell_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    nn.init.zeros_(param)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(embedded)
        output = self.layer_norm(output)
        hidden = self.hidden_norm(
            hidden
        )
        cell = self.cell_norm(cell)
        return output, hidden, cell

In [7]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=4, dropout=0.3):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size
        self.num_directions = 2

        self.embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=23722)

        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout, bidirectional=True)

        self.layer_norm = nn.LayerNorm(2 * hidden_size)
        self.hidden_norm = nn.LayerNorm(hidden_size)
        self.cell_norm = nn.LayerNorm(hidden_size)

        self.fc = nn.Linear(hidden_size * 2, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    nn.init.zeros_(param)

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(1)
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        output = self.layer_norm(output)
        hidden = self.hidden_norm(
            hidden
        )
        cell = self.cell_norm(cell)
        output = self.fc(output).squeeze(1)
        return output, hidden, cell

In [8]:
class Seq2Seq:
    def __init__(self, encoder, decoder, device, teacher_forcing_ratio):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.vocab_size = self.decoder.vocab_size
        self.teacher_forcing_ratio = teacher_forcing_ratio

    def train(
        self,
        train_loader,
        test_loader,
        enc_optimizer,
        dec_optimizer,
        criterion,
        device,
        encoder_scheduler,
        decoder_scheduler,
        num_epochs,
    ):
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            self.encoder.train()
            self.decoder.train()
            epoch_loss = 0
            epoch_acc = 0
            progress_bar = tqdm(train_loader, desc="Training", leave=False)

            for src, trg in progress_bar:
                src, trg = src.to(device), trg.to(device)

                batch_size, trg_len = trg.shape

                enc_optimizer.zero_grad()
                dec_optimizer.zero_grad()

                _, hidden, cell = self.encoder(src)
                outputs = torch.zeros(batch_size, trg_len, self.vocab_size).to(
                    self.device
                )
                x = trg[:, 0]
                for t in range(1, trg_len):
                    output, hidden, cell = self.decoder(x, hidden, cell)
                    outputs[:, t, :] = output
                    teacher_force = torch.rand(1).item() < self.teacher_forcing_ratio
                    x = trg[:, t] if teacher_force else output.argmax(dim=1)
                loss = criterion(
                    outputs[:, 1:].reshape(-1, self.vocab_size), trg[:, 1:].reshape(-1)
                )
                loss.backward()
                enc_optimizer.step()
                dec_optimizer.step()

                epoch_loss += loss.item()

                preds = outputs.argmax(dim=2)
                correct = (preds == trg).float().sum().item()
                total = trg.numel()
                batch_acc = correct / total
                epoch_acc += batch_acc

                progress_bar.set_postfix(
                    loss=f"{loss.item():.4f}", acc=f"{batch_acc:.4f}"
                )

            train_loss = epoch_loss / len(train_loader)
            train_acc = epoch_acc / len(train_loader)

            # ---------------- VALIDATION ---------------- #
            self.encoder.eval()
            self.decoder.eval()
            val_epoch_loss = 0
            val_epoch_acc = 0
            progress_bar = tqdm(test_loader, desc="Validating", leave=False)

            with torch.no_grad():
                for src, trg in progress_bar:
                    src, trg = src.to(device), trg.to(device)
                    batch_size, trg_len = trg.shape
                    _, hidden, cell = self.encoder(src)
                    outputs = torch.zeros(batch_size, trg_len, self.vocab_size).to(
                        self.device
                    )
                    x = trg[:, 0]
                    for t in range(1, trg_len):
                        output, hidden, cell = self.decoder(x, hidden, cell)
                        outputs[:, t, :] = output
                        x = output.argmax(dim=1)

                    loss = criterion(
                        outputs[:, 1:].reshape(-1, self.vocab_size),
                        trg[:, 1:].reshape(-1),
                    )
                    val_epoch_loss += loss.item()

                    preds = outputs.argmax(dim=2)
                    correct = (preds == trg).float().sum().item()
                    total = trg.numel()
                    batch_acc = correct / total
                    val_epoch_acc += batch_acc

                    progress_bar.set_postfix(
                        loss=f"{loss.item():.4f}", acc=f"{batch_acc:.4f}"
                    )

            val_loss = val_epoch_loss / len(test_loader)
            val_acc = val_epoch_acc / len(test_loader)
            print(
                f"Train loss : {train_loss}, Train accuracy : {train_acc}, Val_loss : {val_loss}, val accuracy : {val_acc}"
            )
            encoder_scheduler.step(val_loss)
            decoder_scheduler.step(val_loss)
        return train_loss, train_acc, val_loss, val_acc

    def predict(self, src, trg):
        self.encoder.eval()
        self.decoder.eval()
        with torch.no_grad():
            src, trg = src.to(self.device), trg.to(self.device)
            batch_size, trg_len = trg.shape
            _, hidden, cell = self.encoder(src)
            outputs = torch.zeros(batch_size, trg_len, self.vocab_size).to(self.device)
            x = trg[:, 0]
            for t in range(1, trg_len):
                output, hidden, cell = self.decoder(x, hidden, cell)
                outputs[:, t, :] = output
                x = output.argmax(dim=1)
            return outputs

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)

hidden_size = 256
encoder = Encoder(vocab_size, hidden_size).to(device)
decoder = Decoder(vocab_size, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, device, 0.5)
criterion = nn.CrossEntropyLoss(ignore_index=23722)
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001, weight_decay=1e-5)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001, weight_decay=1e-5)
encoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(encoder_optimizer, mode='min', factor=0.9, patience=2)
decoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(decoder_optimizer, mode='min', factor=0.9, patience=2)

In [10]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

seq2seq.train(train_loader, test_loader, encoder_optimizer, decoder_optimizer, criterion, device, encoder_scheduler, decoder_scheduler, 20)


Epoch 1/20


Train loss : 2.5850319620890496, Train accuracy : 0.5941589877136748, Val_loss : 2.3108001113992875, val accuracy : 0.5887145095870208

Epoch 2/20


Train loss : 1.9146389190967266, Train accuracy : 0.6561314770299149, Val_loss : 2.0735966452455097, val accuracy : 0.6055628533677484

Epoch 3/20


Train loss : 1.5671368112930886, Train accuracy : 0.6836399350071221, Val_loss : 1.8267605557905888, val accuracy : 0.6337420108161258

Epoch 4/20


Train loss : 1.3352552673755549, Train accuracy : 0.7047971198361823, Val_loss : 1.8034481084452265, val accuracy : 0.6410359820550642

Epoch 5/20


...

Train loss : 0.4634282137338932, Train accuracy : 0.8321558938746438, Val_loss : 1.677240277813599, val accuracy : 0.6764726216814158

Epoch 18/20


Train loss : 0.4462985096833645, Train accuracy : 0.8358534766737894, Val_loss : 1.6673911219149564, val accuracy : 0.6771217428711898

Epoch 19/20


Train loss : 0.4335475116968155, Train accuracy : 0.8380074786324785, Val_loss : 1.7000859163503732, val accuracy : 0.6767760570304819

Epoch 20/20


Train loss : 0.40797262283471913, Train accuracy : 0.843958667200855, Val_loss : 1.6996232788119696, val accuracy : 0.6764822240658803


(0.40797262283471913,
 0.843958667200855,
 1.6996232788119696,
 0.6764822240658803)